### Imports


In [1]:
import tensorflow as tf
from platesdetection import ModelLoader
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import numpy as np

2024-05-21 12:38:16.775566: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 12:38:16.775620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 12:38:16.786697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 12:38:16.866025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 12:38:18.539876: W tensorflow/compiler/tf2

In [2]:
model_name = "my_ssd_resnet50_v1_fpn"
#model_name = "my_ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8"
#model_name = "my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet152_v1_fpn_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet101_v1_fpn_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet50_v1_fpn_exported"

images_path = f"TensorFlow/workspace/training_demo/images"

In [3]:
loader = ModelLoader()
teacher_model = loader.load_model(model_name)


[Loading TF2 Checkpoint]
Loading model - my_ssd_resnet50_v1_fpn... 


2024-05-21 12:38:26.890288: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 12:38:26.894499: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 12:38:26.894597: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 12:38:26.925274: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 12:38:26.925431: I external/local_xla/xla/stream_executor

The model has been loaded - 0.32s



#### Dataset

In [4]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    images_path + "/train",
    labels="inferred",
    label_mode="int",
    validation_split=0.2, 
    subset="training",
    seed=1337, 
    image_size=(640, 640),
    batch_size=4
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    images_path + "/test",
    labels="inferred",
    label_mode="int", 
    validation_split=0.2, 
    subset="validation",
    seed=1337, 
    image_size=(640, 640), 
    batch_size=4  
)

Found 409 files belonging to 1 classes.
Using 328 files for training.
Found 22 files belonging to 1 classes.
Using 4 files for validation.


#### Object Detection

In [5]:
@tf.function
def detect_fn(image, detection_model):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

def detect_object(image_np, detection_model, model_name):
    label_filename = "label_map.pbtxt"
    path_to_labels = f"TensorFlow/workspace/training_demo/annotations/{label_filename}"
    category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    if model_name[-9:] == "_exported":
        # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
        input_tensor = tf.convert_to_tensor(image_np)
        # The model expects a batch of images, so add an axis with `tf.newaxis`.
        input_tensor = input_tensor[tf.newaxis, ...]
        detections = detection_model(input_tensor)
    else:
        detections = detect_fn(input_tensor, detection_model)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.30,
            agnostic_mode=False)

    return image_np_with_detections, detections

In [6]:
def create_mobilenet(num_classes):
    model = tf.keras.applications.MobileNetV2(input_shape=[640, 640, 3],
                                                   include_top=False,
                                                   weights=None,
                                                   classes=num_classes)

    model.trainable = True
    return model


def distillation_loss(y_true, y_pred, teacher_preds, alpha=0.1, temperature=3):
    student_loss = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    distillation_loss = tf.keras.losses.KLDivergence()(tf.nn.softmax(teacher_preds / temperature),
                                                      tf.nn.softmax(y_pred / temperature))
    return alpha * student_loss + (1 - alpha) * distillation_loss

def train_student_model(teacher_model, student_model, train_data, val_data, epochs=10, learning_rate=0.001):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    student_model.compile(optimizer=optimizer, loss=distillation_loss)

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        for images, labels in train_data:
            teacher_preds = []
            for image in images:
                detections = detect_fn(tf.convert_to_tensor(np.expand_dims(np.array(image), 0), dtype=tf.float32), teacher_model)
                teacher_preds.append(detections['detection_scores'])

            teacher_preds = tf.convert_to_tensor(teacher_preds)
            student_model.train_on_batch(images, labels, sample_weight=teacher_preds)

        val_loss = 0
        for images, labels in val_data:
            teacher_preds = []
            for image in images:
                detections = detect_fn(tf.convert_to_tensor(np.expand_dims(np.array(image), 0), dtype=tf.float32), teacher_model)
                teacher_preds.append(detections['detection_scores'])

            teacher_preds = tf.convert_to_tensor(teacher_preds)
            val_loss += student_model.test_on_batch(images, labels, sample_weight=teacher_preds)

        print(f'Validation loss: {val_loss / len(val_data)}')

In [7]:
student_model = create_mobilenet(num_classes=1)

In [8]:
train_student_model(teacher_model, student_model, train_data, val_data, epochs=10, learning_rate=0.001)

Epoch 1/10


2024-05-21 12:39:13.604764: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


TypeError: in user code:

    File "/home/hapek-wsl/anaconda3/envs/tf_od/lib/python3.9/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)

    TypeError: tf__distillation_loss() missing 1 required positional argument: 'teacher_preds'
